In [3]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [4]:
olympic_history = "./athlete_events.csv"
o_history_df = pd.read_csv(olympic_history)
o_history_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [5]:
noc_regions = "./noc_regions.csv"
regions_df = pd.read_csv(noc_regions)
regions_df.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [6]:
cols = ["Team","Games", "Season", "City", "Medal", "Sport", "Event"]
olympic_cat_df = o_history_df[cols]
olympic_cat_df.head()


,Team,Games,Season,City,Medal,Sport,Event
0,China,1992 Summer,Summer,Barcelona,NaN,Basketball,Basketball Men's Basketball
1,China,2012 Summer,Summer,London,NaN,Judo,Judo Men's Extra-Lightweight
2,Denmark,1920 Summer,Summer,Antwerpen,NaN,Football,Football Men's Football
3,Denmark/Sweden,1900 Summer,Summer,Paris,Gold,Tug-Of-War,Tug-Of-War Men's Tug-Of-War
4,Netherlands,1988 Winter,Winter,Calgary,NaN,Speed Skating,Speed Skating Women's 500 metres
